## Topics

- How Kafka replication works
- How Kafka handles requests from producers and consumers
- How Kafka handles storage such as file format and indexes

## Cluster Membership

- Kafka uses Apache Zookeeper to maintain the list of brokers that are currently members of a cluster
- Every broker has a unique identifier that is either set in the broker configuration file or automatically generated
- When a broker process starts, it registers itself with its ID in Zookeeper by creating an Zookeeper *ephemeral node*
    - `/brokers/ids` in zookeeper publishes changes when brokers are added or removed
- When a broker loses connectivity to Zookeeper, the ephemeral node that the broker created will be automatically removed from Zookeeper, and Kafka components that are watching the list of brokers will be notified that the broker is gone
- The node representing the broker is gone when the broker is stopped, but the broker ID still exists in other data structures (e.g., list of replicas of each topic, which contains the broker IDs for the replica)
    - So if a brand new broker is started with the ID of a lost broker, it immediately joins the cluster in place of the missing broker with the same partitions and topics assigned to it


## The Controller

Kafka uses Zookeeper's ephemeral node feature to elect a controller and to notify the controller when nodes join and leave the cluster. The controller is responsible for electing leaders among the partitions and replicas whenever it notices nodes join/leave the cluster. The controller uses the epoch number to prevent a "split brain" scenario.

---

- One of the Kafka brokers that is responsible for electing partition leaders, in addition to the usual broker functionality
- The first broker that starts in the cluster becomes the controller by creating an ephemeral node in Zookeeper called `/controller`
- All the following brokers create a *Zookeeper watch* on the controller node so they get notified of changes to this node

---

- When the controller broker is stopped or loses connectivity to Zookeeper, the ephemeral node disappears, and other brokers in the cluster will be notified through the Zookeeper watch that the controller is gone and will attempt to create the controller node in Zookeeper themselves
    - Again the first broker wins, and it receives a new, higher *controller epoch* number through Zookeeper conditional increment operation; so brokers can keep track of the *current* controller
---

- The controller watches the state of other brokers through the relevant Zookeeper path
- If a broker is gone, the controller goes over all the partitions that need a new leader, determines which broker should be the new leader, and sends a request to all the brokers containing those partitions (and the new leader)
    - The request contains information on the new leader and followers for the partitions
    - Each new leader knows that it needs to start serving producer and consumer requests from clients, and the followers know they need to start replicating messages from the new leader

---
- When the controller notices that a broker joined the cluster, it uses the broker ID to check if there are replicas that exist on this broker
    - if so, the controller notifies both new and existing brokers of the change, and the replicas on the new broker start replicating messages from the existing leaders
---



## Replication

- Data in Kafka is organized by topics, each topic is partitioned, and each partition can have multiple replicas
- The replicas are stored on brokers, and each broker typically stores hundreds of thousands of replicas belonging to different topics and partitions

---

- **Leader Replica**: each partition has a single replica designated as the leader; all produce and consume requests go through the leader in order to guarantee consistency
    - Leader is responsible for knowing which of the follower replicas is up-to-date with the leader since followers can fail to stay in sync for reasons like network congestion, crashes, and the such
    - The leader keeps track of the offsets in `Fetch` requests sent by the follower replicas



- **Follower Replica**: all replicas for a partition that are not leaders; followers don't serve client requests, only replicate messages from the leader and stay up-to-date
    - In the event that a leader replica for a partition crashes, one of the follower replicas will be promoted to become the new leader for the partition
    - The follower replicas send the leader `Fetch` requests, much like consumers
    - Replicas who haven't requested a message or has requested messages but hasn't caught up to the most recent messages in more than `replica.lag.time.mx.ms` will be considered as *out of sync*
    - Replicas that are consistently asking for the latest messages are called *in-sync replicas*; only in-sync replicas are eligible to be elected as partition leaders
    
    
---

- *Preferred Leader*: the replica that was the leader when the topic was originally created
    - preferred because when partitions are first created, the leaders are balanced between brokers, such that load will be evenly balanced between brokers
    - `auto.leader.rebalance.enable`: check if the preferred leader replica is not the current leader but is in-sync and trigger leader election to make the preferred leader the current leader

---

## Request Processing

- The partition leaders process requests sent from clients, partition replicas, and the controller
- Kafka uses a binary protocol over TCP 
- All requests sent to the broker from a specific client will be processed in the order in which they were received - providing ordering guarantees

---

### Request Header

- Request type: also called API key
- Request version: for compatibility
- Correlation ID: a number that uniquely identifies the request and also appears in the response and in the error logs (troubleshoot)
- Client ID: used to identify the application that sent the request

---

### Request Handling

<img src="img/Snip20200429_1.png" width=80%/>

- For each port the broker listens on, the broker runs an *acceptor* thread that creates a connection and hands it over to a *processor* thread for handling
    - processor threads are also called *network thread*, whose number is configurable


- The network threads are responsible for taking requests from client connections, placing them in a *request queue*, and picking up responses from a *response queue* and sending them back to clients


- Once requests are placed on the request queue, *IO threads* are responsible for picking them up and processing them


- Kafka clients are responsible for sending produce and fetch requests to the broker that contains the leader for the relevant partition for the request; failing to do so will result in the broker returning an error response for not being the leader
    - Kafka clients also send *metadata request*, which includes a list of topics the client is interested in, to get metadata about which partitions exist in the topics, the replicas for each partition, and which replica is the leader
    - Metadata are cached on client side and on each broker, clients refresh its cached metadata every `meta.data.max.age.ms`, also when it receives a "Not a leader" error to one of its requests

<img src="img/Snip20200429_2.png" width=80%/>

---

### Produce Requests

- `acks` controls the number of brokers who need to acknowledge receiving the message before it is considered a successful write
    - `acks=0`: message is sent without waiting for the broker to accept it at all
    - `acks=1`: just the leader
    - `acks=all`: all in-sync replicas
    
    
- A leader broker performs validations on a certain message it receives to decide whether to accept the message or not
    - including permission checks, request parameter checks, and in-sync replicas checks (optionally refuse a message when there is not enough in-sync replicas)


- After the message is written to the leader, the broker takes actions depending on the `acks` configuration:
    - `0` or `1`: respond immediately
    - `all`:  the request is stored in a buffer called *purgatory* until the leader observes that the follower replicas replicated the message, at which point a response is sent to the client

---

### Fetch Requests

- Requests client sent to ask for messages from a list of topics, partitions, and offsets
- Clients specify a **upper boundary to how much data the broker can return** for each partition, to avoid out-of-memory situation
- Clients can also specify a **lower boundary on the amount of data**, to reduce CPU and network utilization when clients are reading from topics that are not seeing much traffic
- Clients can define a **timeout** such that the delayed delivery would not be postponed forever
- Fetch request has to arrive to the leaders of the partitions, the client ensure such routing using the metadata from the metadata request

---

- Kafka uses a `zero-copy` method to send the messages to the clients: which means that Kafka sends messages from the file directly to the network channel without any intermediate buffers, reducing the overhead of copying bytes and managing buffers in the memory and resulting in much improved performance

---

- Clients can only read messages that were written to all in-sync replicas
    - The leader of the partition knows which messages were replicated to which replica, and until a message was written to all in-sync replicas, it will not be sent to consumers
    - This is done to guarantee that the messages consumed are indeed replicated and persisted
    
<img src="img/Snip20200429_3.png" width=80%/>

## Physical Storage

- The basic storage unit of Kafka is a partition replica
    - The size of a partition is limited by the space available on a single mount point, because it cannot be split between multiple brokers or multiple disks on the same broker
    

- `log.dirs`: defines a list of directories in which the partitions will be stored
    - not to be confused with where Kafka stores its error logs, which is configured through the `log4j.properties` file


### Partition Allocation

- When a topic is created, Kafka decides how to allocate the partitions between brokers, goals are
    - To spread replicas evenly among brokers
    - To make sure that for each partition, each replica is on a different broker
    - If the brokers have rack information, then assign the replicas for each partition to different racks if possible


- The allocation of partitions to brokers does not take available space or existing load into account
- The allocation of partitions to disks does not take the size of the partitions into account, only the number of partitions
    - Be careful with the partition allocation if some partitions are abnormally large, or disks have different sizes on the same broker


### File Management

- Kafka can be configured a retention period for each topic: either the amount of time to store messages before deleting them, or how much data to store before older messages are purged


- Kafka split each partition in to segments; by default, each segment contains either 1 GB of data or a week of data, whichever is smaller; as a broker is writing to a partition, if the segment limit is reached, the file is closed and a new one is created
    - The segment currently being written to is called an *active segment*, which is never deleted
    - A Kafka broker keeps an open file handle to every segment in every partition, evene inactive ones

### File Format

- Each segment is stored in a single data file, insdie which stores the Kafka messages and offsets
- The format of the data on the disk is identical to the format of the messages that were sent from the producer; the same format is used when the messages are sent to consumers
    - This allows Kafka brokers to use zero-copy optimization when sending messages to consumers; also avoid decompressing and recompressing messages 



- Each message contains its **key, value, offset**, and things like **the message size**, **checksum code** for corruption detection, **magic byte** that indicates the version of the message format, **compression codec** (Snappy, GZip, or LZ4), and a **timestamp**



- If the producer is sending compressed messages, all the messages in a single producer batch are compressed together and sent as the "value" of a "wrapper message"
    - If compression is used on the producer side, then sending larger batches means better compression both over the wire and on the broker disks

<img src="img/Snip20200502_1.png" width=80%/>



- Kafka brokers ship with the `DumpLogSegment` tool which allos inspection on a particular segment in the filesystem
    - `bin/kafka-run-class.sh kafka.tools.DumpLogSegments`
        - optional `--deep-iteration` param to show messages compressed inside the wrapper messages



### Indexes

- Kafka allows consumers to start fetching messages from any available offset; which is done efficiently by Kafka by maintining an index for each partition
    - The index maps offsets to segment files and positions within the file
- Indexes are broken into segments, so old index entries can be deleted when the messages are purged
- Indexes are safe to regenerate as they can be built from re-reading the messages and recording the offsets and locations

### Compaction

- Sample application: tracking a specific state, such as a customer's address, where you only care about the latest update and nothing else
- Kafka supports log compaction by allowing the retention policy on a topic to be *delete*, which delete events older than retention time, to *compact*, which only stores the most recent value for each key in the topic


- Compaction is enabled using the `log.cleaner.enabled` config
- In implementation, each log is viewed as split into two portions, **clean** and **dirty**
    - `Clean`: messages that have been compacted before; which contains only one value for each key: the latest value at the time of the previous compaction
    - `Dirty`: messages that were written after the last compaction
    
<img src="img/Snip20200502_2.png" width=80%/>

- Each broker starts a compaction manager thread and a number of compaction threads
- To compact a partition, the cleaner thread reads the dirty section of the partition and creates an in-memory map; each map entry contains a 16-byte hash of a message key and the 8-byte offset of the previous message that had the same key
- The memory usage of compaction threads' offset maps can be limited in configuration
    - Kafka does not require the entire dirty section of the partition to fit into the size allocated for the offset map, but at least one full segment has to fit
- Once the offset map is built, the cleaner thread will start reading off the clean segments, starting with the oldest, and check their contents against the offset map to see whether if the key exists in the offset map
    - If the key does not exist, the value remains the latest and the message is copies over to a replacement segment
    - If the key does exist, the messages is omitted
- Once all latest messages are copied, the replacement segment and the original segment are swapped

<img src="img/Snip20200502_3.png" width=80%/>


### Deleted Events

- In cases where we want to delete all messages for a specific key, the application must produce a message that contains the key and a null value
- When the cleaner thread finds such a message, it will first do a normal compaction and retain only the message with the null value; this special message, known as a *tombstone*, is kept around for a configurable amount of time
- Consumers will be able to see the tombstone message and know that the value is deleted; so that the downstream systems can perform the corresponding actions such as deleting data from a relational database
- It is important to give consumers enough time to see the tombstone message such that it will not be missed 